In [ ]:
import os
import numpy as np
import pandas as pd
import re
import io
import json
from pandas.io.json import json_normalize
import csv
import random
import matplotlib.pyplot as plt
import datetime
from scipy import stats
from datetime import datetime
import seaborn as sns
import pickle

from scipy.interpolate import UnivariateSpline
from sklearn.metrics import roc_curve, auc
from scipy.signal import find_peaks

pd.set_option('display.max_colwidth', None)

In [ ]:
# Sample notebook from ECG deep learning project
from sklearn.model_selection import GroupKFold, KFold

import tensorflow as tf  

from tensorflow import keras

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Reshape
from tensorflow.keras.layers import Dropout, Lambda
from tensorflow.keras.layers import Conv1D, Conv2D, Conv2DTranspose,Convolution2D, Conv3D, ConvLSTM2D, Bidirectional
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dense

from tensorflow.keras import regularizers
from tensorflow.keras import optimizers

from tensorflow.keras.layers import Activation, SpatialDropout1D, SpatialDropout2D, LayerNormalization, GlobalAvgPool1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import Callback

import tensorflow.keras.backend as K

from tensorflow_addons.layers import WeightNormalization
from tensorflow_addons.optimizers import AdamW 

In [ ]:
#Opening numpy arrays for sex    
suffix = 'corrected_60k_sex'           #non-null- 56,665 ECGs; will use 53,665 in training set

ecg_X_k = np.load('MGH_60k_500hz_ecgs_age_18_100_trainval_X.npy')
ecg_y_k = np.load('MGH_60k_500hz_ecgs_age_18_100_trainval_yfemale.npy')

ecg_X_k = ecg_X_k[ecg_y_k>=0]
ecg_y_k = ecg_y_k[ecg_y_k>=0]

print(ecg_X_k.shape)
print(ecg_y_k.shape)

np.unique(ecg_y_k)

In [ ]:
#opening numpy arrays for sex test set

ecg_X_k = np.load('MGH_60k_500hz_ecgs_age_18_100_test_X.npy')
ecg_y_k = np.load('MGH_60k_500hz_ecgs_age_18_100_test_y.npy')

ecg_X_k = ecg_X_k[ecg_y_k>=0]
ecg_y_k = ecg_y_k[ecg_y_k>=0]

print(ecg_X_k.shape)
print(ecg_y_k.shape)

np.unique(ecg_y_k)

In [ ]:
#Opening numpy arrays for age    #57,000; 54,150 in training with 20-fold
suffix = 'corrected_60k_age'

ecg_X_k = np.load('MGH_60k_500hz_ecgs_age_18_100_trainval_X.npy')
ecg_y_k = np.load('MGH_60k_500hz_ecgs_age_18_100_trainval_yage.npy')

In [ ]:
ecg_X_k = ecg_X_k[ecg_y_k>0]
ecg_y_k = ecg_y_k[ecg_y_k>0]

In [ ]:
print(ecg_X_k.shape)
print(ecg_y_k.shape)

In [ ]:
#selecting only X and y elements in 60k dataset where rhythm is sinus

In [ ]:
trainval_sinus = np.load('MGH_60k_500hz_ecgs_age_18_100_trainval_sinus.npy')

In [ ]:
#get measurements

In [ ]:
ecg_filename_k = np.load('MGH_60k_500hz_ecgs_age_18_100_trainval_filename.npy')

In [ ]:
ecg_metadata = []

for file in ecg_filename_k:
    
    
    with open('/mnt/obi0/phi/ecg/convertedData/MGH/' + file[0:5] + '/' + file + '.dict', 'rb') as f:
        ecg_dict = pickle.load(f)

    try:
        ecg_dict['metadata']['StmtText'] = ' '.join([str(elem) for elem in ecg_dict['metadata']['StmtText']])
        ecg_dict['metadata']['Filename'] = file
        ecg_metadata.append(ecg_dict['metadata'])
        
    except:
        ecg_metadata.append({'Filename':file})

meta_df = pd.DataFrame.from_dict(ecg_metadata)

In [ ]:
meta_df.shape

In [ ]:
meta_df = meta_df[ecg_y_k>0]


In [ ]:
meta_df = meta_df[trainval_sinus==True]

In [ ]:
ecg_X_k = ecg_X_k[trainval_sinus==True]
ecg_y_k = ecg_y_k[trainval_sinus==True]

print(ecg_X_k.shape)          #27,215 in trainval set; 15 folds used -> 25,400 for training set
print(ecg_y_k.shape)

In [ ]:
meta_df.shape

In [ ]:
meta_df[['VentricularRate', 'AtrialRate', 'PRInterval', 'QRSDuration', 'QTInterval', 'QTCorrected', 'PAxis', 'RAxis', 'TAxis', 'QRSCount', 'QOnset', 'QOffset', 'POnset', 'POffset', 'TOffset']] = meta_df[['VentricularRate', 'AtrialRate', 'PRInterval', 'QRSDuration', 'QTInterval', 'QTCorrected', 'PAxis', 'RAxis', 'TAxis', 'QRSCount', 'QOnset', 'QOffset', 'POnset', 'POffset', 'TOffset']].apply(pd.to_numeric)

In [ ]:
meta_df[['QOnset', 'QOffset', 'POnset', 'POffset', 'TOffset']] = meta_df[['QOnset', 'QOffset', 'POnset', 'POffset', 'TOffset']] *2

In [ ]:
meta_df.iloc[0:5, 10:20]

In [ ]:
meta_df.head(2)

In [ ]:
meta_df.isna().sum()    #some missing atrial rate (52), p axis (182), qrs count (52), p onset (125), p offset (125)

In [ ]:
meta_df_nonull = meta_df[pd.notnull(meta_df['POnset'])]

In [ ]:
meta_df_nonull.shape

In [ ]:
ecg_X_k = ecg_X_k[pd.notnull(meta_df['POnset'])]   #27,090, will use 24,090 for training
ecg_y_k = ecg_y_k[pd.notnull(meta_df['POnset'])]

print(ecg_X_k.shape)         
print(ecg_y_k.shape)

In [ ]:
meta_df_set = meta_df_nonull[['VentricularRate', 'PRInterval', 'QRSDuration', 'QTInterval', 'QTCorrected', 'RAxis', 'TAxis', 'QOnset', 'QOffset', 'TOffset']]

In [ ]:
meta_df_set.shape

In [ ]:
meta_df_set.isna().sum() 

In [ ]:
#To get only lead I
ecg_X_k = ecg_X_k[:, 0:5000, :]

In [ ]:
#To get 2D array for 12-lead ecgs
ecg_X_k = ecg_X_k.reshape(ecg_X_k.shape[0], 12, int(ecg_X_k.shape[1]/12), ecg_X_k.shape[2])

In [ ]:
ecg_X_k = ecg_X_k.reshape(ecg_X_k.shape[0], ecg_X_k.shape[1])   #removes last dimension

In [ ]:
ecg_X_rescaled = np.zeros((ecg_X_k.shape))

In [ ]:
for ecgnum in range(ecg_X_k.shape[0]):            #normalizes with mean 0 and max 1
    ecg_X_rescaled[ecgnum] = ecg_X_k[ecgnum]-np.mean(ecg_X_k[ecgnum])
    ecg_X_rescaled[ecgnum] = ecg_X_rescaled[ecgnum]/np.max(ecg_X_rescaled[ecgnum])


In [ ]:
skipped1 = []
skipped2 = []
skipped3 = []
ecg_X_full_intervals = np.zeros((0, 6))
ecg_X_intervals = np.zeros((0, 6))

for ecg_num in range(ecg_X_rescaled.shape[0]):
    peaks, _ = find_peaks(ecg_X_rescaled[ecg_num], distance = 150, prominence=0.6)
    
    if (len(peaks)>=7):
        if (peaks[6]-peaks[0]>= 1500) & (peaks[6]-peaks[0]<=4500):  #selects mean HR 40-120 for the 7 beats
        
            #finds R-R intervals, as the percentage of the 7 beat group for each R-R interval
            rr_ints = [(t - s)/(peaks[6]-peaks[0]) for s, t in zip(peaks[0:6], peaks[1:7])] 
            rr_full = [(t - s) for s, t in zip(peaks[0:6], peaks[1:7])] 
        
            ecg_X_full_intervals = np.concatenate((ecg_X_full_intervals, np.array([rr_full])))
            ecg_X_intervals = np.concatenate((ecg_X_intervals, np.array([rr_ints])))
        else: 
            if (peaks[6]-peaks[0]< 1500):
                skipped1.append(ecg_num)
            if (peaks[6]-peaks[0]>4500):
                skipped2.append(ecg_num)
    else:
        skipped3.append(ecg_num)                

In [ ]:
len(skipped1)       # 614 skipped for detected mean HR for first 7 beats >120 (oversensing t-wave?)

In [ ]:
len(skipped2)       # 11 skipped for detected mean HR for first 7 beats <40

In [ ]:
len(skipped3)       # 284 skipped for not having at least 7 beats detected

In [ ]:
skipped = skipped1 + skipped2 + skipped3

In [ ]:
len(skipped)        # 909

In [ ]:
ecg_X_intervals.shape   #26,306 kept

In [ ]:
ecg_X_full_intervals.shape

In [ ]:
ecg_X_intervals

In [ ]:
ecg_X_full_intervals

In [ ]:
mask = np.ones_like(ecg_y_k, bool)

In [ ]:
mask[skipped]=False

In [ ]:
ecg_y_intervals = ecg_y_k[mask]       #26,306 in y numpy array
print(ecg_y_intervals.shape)

In [ ]:
np.mean(ecg_X_intervals)  #0.167

In [ ]:
np.max(ecg_X_intervals)   #0.509

In [ ]:
np.min(ecg_X_intervals)   #0.042

In [ ]:
ecg_X_intervals_all = ecg_X_intervals.flatten()

In [ ]:
plt.hist(ecg_X_intervals_all)
plt.yscale('log')

In [ ]:
#will set min and max R-R intervals to avoid PACs and PVCs
#mean R-R: 0.1667
#max- 0.200 : 20% increase
#min- 0.133 : 20% decrease
#difference between min and max- 50%
ecg_y_intervals = ecg_y_intervals[(np.min(ecg_X_intervals, axis=1)>=0.133) & (np.max(ecg_X_intervals, axis=1)<=0.2)]
ecg_X_intervals = ecg_X_intervals[(np.min(ecg_X_intervals, axis=1)>=0.133) & (np.max(ecg_X_intervals, axis=1)<=0.2),:]

print(ecg_X_intervals.shape)    #23,509 of 26,306 remaining; will use 2,000 for val set
print(ecg_y_intervals.shape)

In [ ]:
ecg_X_full_intervals = ecg_X_full_intervals[(np.min(ecg_X_intervals, axis=1)>=0.133) & (np.max(ecg_X_intervals, axis=1)<=0.2),:]


In [ ]:
ecg_X_full_intervals.shape

In [ ]:
X_train_full_list = [0]
X_train_full_list[0] = ecg_X_full_intervals[0:21509]

In [ ]:
#alternative to k-fold 

X_train_list = [0]
y_train_list = [0]
X_val_list = [0]
y_val_list = [0]

X_train_list[0] = ecg_X_intervals[0:24090]
y_train_list[0] = ecg_y_intervals[0:24090]

X_val_list[0] = ecg_X_intervals[24090:]
y_val_list[0] = ecg_y_intervals[24090:]

In [ ]:
#to visualize ECGs
ecg_num= 108

peaks, _ = find_peaks(ecg_X_rescaled[ecg_num], distance = 150, prominence=0.6) 

plt.figure(figsize=(14,10))
plt.plot(ecg_X_rescaled[ecg_num])
plt.plot(peaks, ecg_X_rescaled[ecg_num][peaks], 'x')

In [ ]:
X_train_list = [0]
y_train_list = [0]
X_val_list = [0]
y_val_list = [0]

X_train_list[0] = ecg_X_k[0:21509]
y_train_list[0] = ecg_y_k[0:21509]

X_val_list[0] = ecg_X_k[21509:]
y_val_list[0] = ecg_y_k[21509:]

In [ ]:

X_train_list = [0]
y_train_list = [0]
X_val_list = [0]
y_val_list = [0]
meta_df_train_list = [0]
meta_df_val_list = [0]

X_train_list[0] = ecg_X_k[0:24090]
y_train_list[0] = ecg_y_k[0:24090]
meta_df_train_list[0] = meta_df_set[0:24090]

X_val_list[0] = ecg_X_k[24090:]
y_val_list[0] = ecg_y_k[24090:]
meta_df_val_list[0] = meta_df_set[24090:]

In [ ]:
X_train_list[0].shape

In [ ]:
X_train_list[0] = X_train_list[0][:, 0:5000, :]

In [ ]:
X_val_list[0] = X_val_list[0][:, 0:5000, :]

In [ ]:
y_val_list[0].shape

In [ ]:
np.unique(y_val_list)

In [ ]:
#if reshaping needed
X_train_list[0] = X_train_list[0].reshape(X_train_list[0].shape[0], 12, 5000, 1)
X_val_list[0] = X_val_list[0].reshape(X_val_list[0].shape[0], 12, 5000, 1)

In [ ]:
X_val_list[0][0][0:10]

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(X_val_list[0][45][0:500])

In [ ]:
#from https://www.jeremyjordan.me/nn-learning-rate/

class LRFinder(keras.callbacks.Callback):
    
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        plt.show()
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.show()

In [ ]:
#from https://github.com/psklight/keras_one_cycle_clr/blob/master/keras_one_cycle_clr/one_cycle.py

class OneCycle(keras.callbacks.Callback):
    """
    A callback class for one-cycle policy training.
    :param lr_range: a tuple of starting (usually minimum) lr value and maximum (peak) lr value.
    :param momentum_range: a tuple of momentum values.
    :param phase_one_fraction: a fraction for phase I (increasing lr) in one cycle. Must between 0 to 1.
    :param reset_on_train_begin: True or False to reset counters when training begins.
    :param record_frq: integer > 0, a frequency in batches to record training loss.
    :param verbose: True or False to print progress.
    """

    def __init__(
            self,
            lr_range,
            momentum_range=None,
            phase_one_fraction=0.3,
            reset_on_train_begin=True,
            record_frq=10,
            verbose=False):

        super(OneCycle, self).__init__()

        self.lr_range = lr_range

        self.momentum_range = momentum_range
        if momentum_range is not None:
            err_msg = "momentum_range must be a 2-numeric tuple (m1, m2)."
            if not isinstance(momentum_range, (tuple,)) or len(momentum_range) != 2:
                raise ValueError(err_msg)

        self.phase_one_fraction = phase_one_fraction
        self.reset_on_train_begin = reset_on_train_begin
        self.record_frq = record_frq
        self.verbose = verbose

        # helper tracker
        self.log = {}  # history in iterations
        self.log_ep = {}  # history in epochs
        self.stop_training = False

        # counter
        self.current_iter = 0

    def get_current_lr(self, n_iter=None):
        """
        A helper function to calculate a current learning rate based on current iteration number.
        :return lr: a current learning rate.
        """
        if n_iter is None:
            n_iter = self.n_iter

        x = float(self.current_iter) / n_iter
        if x < self.phase_one_fraction:
            amp = self.lr_range[1] - self.lr_range[0]
            lr = (np.cos(x * np.pi/self.phase_one_fraction - np.pi) + 1) * amp / 2.0 + self.lr_range[0]
        if x >= self.phase_one_fraction:
            amp = self.lr_range[1]
            lr = (np.cos((x - self.phase_one_fraction) * np.pi/ (1-self.phase_one_fraction)) + 1) / 2.0 * amp
        return lr

    def get_current_momentum(self, n_iter=None):
        """
        A helper function to calculate a current momentum based on current iteration number.
        :return momentum: a current momentum.
        """
        if n_iter is None:
            n_iter = self.n_iter
        amp = self.momentum_range[1] - self.momentum_range[0]
        # delta = (1 - np.abs(np.mod(self.current_iter, n_iter) * 2.0 / n_iter - 1)) * amplitude
        x = float(self.current_iter) / n_iter
        if x < self.phase_one_fraction:
            delta = (np.cos(x * np.pi / self.phase_one_fraction - np.pi) + 1) * amp / 2.0
        if x >= self.phase_one_fraction:
            delta = (np.cos((x - self.phase_one_fraction) * np.pi / (1 - self.phase_one_fraction)) + 1) / 2.0 * amp
        return delta + self.momentum_range[0]


    @property
    def cycle_momentum(self):
        return self.momentum_range is not None

    def on_train_begin(self, logs={}):
        self.n_epoch = self.params['epochs']

        # find number of batches per epoch
        if self.params['batch_size'] is not None:  # model.fit
            self.n_bpe = int(np.ceil(self.params['samples'] / self.params['batch_size']))
        if self.params['batch_size'] is None:  # model.fit_generator
            self.n_bpe = self.params['samples']

        self.n_iter = self.n_epoch * self.n_bpe
        # this is a number of iteration in one cycle

        self.current_iter = 0

    def on_train_batch_begin(self, batch, logs={}):
        set_lr(self.model.optimizer, self.get_current_lr())
        if self.cycle_momentum:
            set_momentum(self.model.optimizer, self.get_current_momentum())

    def on_train_batch_end(self, batch, logs={}):

        if self.verbose:
            print("lr={:.2e}".format(self.get_current_lr()), ",", "m={:.2e}".format(self.get_current_momentum()))

        # record according to record_frq
        if np.mod(int(self.current_iter), self.record_frq) == 0:
            self.log.setdefault('lr', []).append(self.get_current_lr())
            if self.cycle_momentum:
                self.log.setdefault('momentum', []).append(self.get_current_momentum())

            for k, v in logs.items():
                self.log.setdefault(k, []).append(v)

            self.log.setdefault('iter', []).append(self.current_iter)

        # update current iteration
        self.current_iter += 1

        # consider termination
        if self.current_iter == self.n_iter:
            self.stop_training = True

    def on_epoch_end(self, epoch, logs={}):
        self.log_ep.setdefault('epoch', []).append(epoch)
        self.log_ep.setdefault('lr', []).append(
            K.get_value(self.model.optimizer.lr))

        for k, v in logs.items():
            self.log_ep.setdefault(k, []).append(v)

    def test_run(self, n_iter=None):
        """
        Visualize values of learning rate (and momentum) as a function of iteration (batch).
        :param n_iter: a number of cycles. If None, 1000 is used.
        """

        if hasattr(self, 'current_iter'):
            original_it = self.current_iter

        if n_iter is None:
            if hasattr(self, 'n_iter'):
                n_iter = self.n_iter
            else:
                n_iter = 1000
        n_iter = int(n_iter)

        lrs = np.zeros(shape=(n_iter,))
        if self.momentum_range is not None:
            moms = np.zeros_like(lrs)

        for i in range(int(n_iter)):
            self.current_iter = i
            lrs[i] = self.get_current_lr(n_iter)
            if self.cycle_momentum:
                moms[i] = self.get_current_momentum(n_iter)
        if not self.cycle_momentum:
            plt.plot(lrs)
            plt.xlabel('iterations')
            plt.ylabel('lr')
        else:
            plt.figure(figsize=(10, 4))
            plt.subplot(1, 2, 1)
            plt.plot(lrs)
            plt.xlabel('iterations')
            plt.ylabel('lr')
            plt.subplot(1, 2, 2)
            plt.plot(moms)
            plt.xlabel('iterations')
            plt.ylabel('momentum')

        if hasattr(self, 'current_iter'):
            self.current_iter = original_it

def set_momentum(optimizer, mom_val):
    """
    Helper to set momentum of Keras optimizers.
    :param optimizer: Keras optimizer
    :param mom_val: value of momentum.
    """
    keys = dir(optimizer)
    if "momentum" in keys:
        K.set_value(optimizer.momentum, mom_val)
    if "rho" in keys:
        K.set_value(optimizer.rho, mom_val)
    if "beta_1" in keys:
        K.set_value(optimizer.beta_1, mom_val)


def set_lr(optimizer, lr):
    """
    Helper to set learning rate of Keras optimizers.
    :param optimizer: Keras optimizer
    :param lr: value of learning rate.
    """
    K.set_value(optimizer.lr, lr)

In [ ]:
X_train_list[0] = ecg_X_k[0:24090]
y_train_list[0] = ecg_y_k[0:24090]
meta_df_train_list[0] = meta_df_set[0:24090]

In [ ]:
meta_df_train_list[0].head()

In [ ]:
meta_df_train = meta_df_train_list[0].to_numpy()

In [ ]:
meta_df_val = meta_df_val_list[0].to_numpy()

In [ ]:
meta_df_val.shape

In [ ]:
#Regression (binary classification separate)
#suffix = 'sex_8k'
#suffix = 'pr_12lead_2d'

In [ ]:
#suffix = 'corrected_60k_age_12lead'

In [ ]:
suffix

In [ ]:
K.clear_session()
filename_base = 'trained_' + suffix + '_1'
print(filename_base)

in_neurons=5000

In [ ]:
#12-lead model
inputs = Input((12, in_neurons, 1))

c1 = Conv2D(16,(1,4), padding='same', strides=(1,2), use_bias = False, kernel_initializer= 'he_normal') (inputs)
c1 = BatchNormalization() (c1)
c1 = Activation('relu') (c1)
c1 = Dropout(0.1) (c1)

c1 = Conv2D(32,(1,8), padding='same', strides=(1,2), use_bias = False, kernel_initializer= 'he_normal') (c1)
c1 = BatchNormalization() (c1)
c1 = Activation('relu') (c1)
c1 = Dropout(0.1) (c1)

s1 = Conv2D(16, (1,8), padding='same', use_bias=False, kernel_initializer= 'he_normal') (c1)
s1 = BatchNormalization() (s1)

c2 = WeightNormalization(Conv2D(32,(1,8), padding='same', dilation_rate=(1,2), kernel_initializer= 'he_normal')) (c1)
c2 = Activation('relu') (c2)
c2 = SpatialDropout2D(0.1) (c2)
c2 = WeightNormalization(Conv2D(32,(1,8), padding='same', dilation_rate=(1,4), kernel_initializer= 'he_normal')) (c2)
#c2 = Activation('relu') (c2)
c2 = Activation('relu') (c1+c2)
c2 = SpatialDropout2D(0.1) (c2)
#c2 = MaxPooling1D(pool_size=1, strides=4) (c2)

c3 = WeightNormalization(Conv2D(16,(1,8), padding='same', dilation_rate=(1,8), kernel_initializer= 'he_normal')) (c2)
c3 = Activation('relu') (c3)
c3 = SpatialDropout2D(0.1) (c3)
c3 = WeightNormalization(Conv2D(16,(1,8), padding='same', dilation_rate=(1,16), kernel_initializer= 'he_normal')) (c3)
#c3 = Activation('relu') (c3)
c3 = Activation('relu') (s1+c3)
c3 = SpatialDropout2D(0.1) (c3)
c3 = MaxPooling2D(pool_size=1, strides=(1,8)) (c3)

c7 = Conv2D(8, (1,3), use_bias = False, kernel_initializer= 'he_normal') (c3)
c7 = BatchNormalization() (c7)
c7 = Activation('relu') (c7)
c7 = Dropout(0.3) (c7)

c7 = Conv2D(16, (12,1), use_bias=False, kernel_initializer='he_normal') (c7)
c7 = BatchNormalization() (c7)
c7 = Activation('relu') (c7)
c7 = Dropout(0.3) (c7)

final = Flatten() (c7)

outputs = Dense(1) (final)

In [ ]:
#changed to Adam, changed batch size

set_loss = 'mean_squared_error'
set_metrics = ['mean_squared_error'] 

num_batch_size=512   #180
num_epochs=100
num_patience=30
stopping_min_epochs=80
weight_d = 0 #0.000001  

#adamw = AdamW(weight_decay=weight_d)
adam = optimizers.Adam()

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer= adam, loss = set_loss, metrics= set_metrics)
model.save_weights('temp4.h5')

model.summary()

In [ ]:
#ocp = OneCycle(lr_range=(0.0007, 0.007), momentum_range=(0.94,0.85))
#ocp.test_run(128)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
i = 0
#y_train_list_scaled = y_train_list[i]
#y_val_list_scaled = y_val_list[i]

In [ ]:
#removed age scaling

#X_train_list_scaled = np.multiply(np.sign(X_train_list[0]), np.log(np.abs(X_train_list[0])+1)) #log-modulus transform- https://blogs.sas.com/content/iml/2014/07/14/log-transformation-of-pos-neg.html
#X_val_list_scaled = np.multiply(np.sign(X_val_list[0]), np.log(np.abs(X_val_list[0])+1))

#12-lead- too large
#X_train_list_scaled_a = np.divide(X_train_list[i][0:27000],4000)
#X_train_list_scaled_b = np.divide(X_train_list[i][27000:],4000)
#X_train_list_scaled = np.vstack((X_train_list_scaled_a, X_train_list_scaled_b))


X_train_list_scaled = np.divide(X_train_list[i], 4000)
X_val_list_scaled = np.divide(X_val_list[i], 4000)

y_train_list_scaled = np.multiply((y_train_list[i]), 4)   #for age
y_val_list_scaled = np.multiply((y_val_list[i]), 4)

#padding to replicate Mayo Clinic paper
#X_train_list_scaled = np.pad(X_train_list_scaled, pad_width=((0,0),(0,0),(60,60),(0,0)))
#X_val_list_scaled = np.pad(X_val_list_scaled, pad_width=((0,0),(0,0),(60,60),(0,0)))


#sns.distplot(X_train_list_scaled)

#Normalization
#X_train_list_scaled = np.log(X_train_list[0])
#print(np.mean(X_train_list_scaled))
#print(np.std(X_train_list_scaled))

#X_val_list_scaled = (X_val_list[0]-np.mean(X_val_list[0]))/370
#print(np.mean(X_val_list_scaled))
#print(np.std(X_val_list_scaled))

In [ ]:
X_train_list_scaled.shape

In [ ]:
y_train_list_scaled.shape

In [ ]:
#X_train_list_lr = X_train_list_scaled[0:5000]

In [ ]:
#y_train_list_lr = y_train_list_scaled[0:5000]

In [ ]:
#lr_finder = LRFinder(min_lr=1e-4, max_lr=1e-1, steps_per_epoch=np.ceil(X_train_list_scaled.shape[0]/num_batch_size), epochs=1)

#model.fit(X_train_list_scaled, y_train_list_scaled, callbacks=[lr_finder])
#lr_finder.plot_loss()

In [ ]:
#changed to Adam

best_epochs = []
mses = []
val_mses = []
mse_hists = []
val_mse_hists = []
spearmans = []
pearsons = []
mean_aes = []
median_aes = []

#for train_index, val_index in group_kfold.split(ecg_X_k, ecg_y_k, ecg_id_k):  #for full 10-fold cross validation


for j in range(i, (i+1)):
    
    K.clear_session()

    adamw = AdamW(weight_decay=weight_d)
    adam = optimizers.Adam()  #, clipnorm=1.)
    #rmsprop = optimizers.RMSprop(learning_rate= set_learning_rate, clipnorm=1.)
    #sgdopt = optimizers.SGD(learning_rate= set_learning_rate, clipnorm=1.)
    #reduce_lr = tf.keras.callbacks.ReduceLROnPlateau()
    
    
    
    model = Model(inputs=[inputs], outputs=[outputs])
    #model.compile(optimizer= adamw, loss = set_loss, metrics= set_metrics)
    model.compile(optimizer= adam, loss = set_loss, metrics= set_metrics)
    model.load_weights('temp4.h5')
    #model = load_model(filename_base + '_fold_' + str(i) + '.h5', custom_objects={'AdamW':adamw})
    
    
    #corr_check = corr_checkpointer(X_val_list_scaled, y_val_list_scaled, filepath= (filename_base + '_fold_' + str(i) + '.h5'))
    #early_stopper = corr_early_stopping(min_epochs=stopping_min_epochs, mode='max', patience=num_patience, verbose=1)
    
    onecyc = OneCycle(lr_range=(0.001, 0.01), momentum_range=(0.95,0.85))
    
    earlystopper = EarlyStopping(monitor = 'val_mean_squared_error', mode='min', patience=num_patience, verbose=1)  #restore_best_weights not available
    checkpointer = ModelCheckpoint(filename_base + '_fold_' + str(i) + '.h5', monitor = 'val_mean_squared_error', mode='min', verbose=1, save_best_only = True)


    
    #reduce_lr = tf.keras.callbacks.ReduceLROnPlateau()


    history = model.fit(X_train_list_scaled, y_train_list_scaled, validation_data = (X_val_list_scaled, y_val_list_scaled), batch_size = num_batch_size, epochs=num_epochs, callbacks=[checkpointer, onecyc, earlystopper])   #[corr_check, onecyc, early_stopper]) #tensorboard_callback])
    


    

In [ ]:
now = datetime.now()

current_time2 = now.strftime("%H:%M:%S")
print("Start time: ", current_time)
print("End time: ", current_time2)

In [ ]:
#should be in above loop
#filename_base = 'trained_' + suffix + '_12'
#adamw = AdamW(weight_decay=weight_d)

#best_epoch = len(history.history['val_mean_squared_error']) - num_patience
#mse_model = history.history['mean_squared_error'][-(num_patience + 1)]
#val_mse_model = history.history['val_mean_squared_error'][-(num_patience + 1)]
#mse_hist = history.history['mean_squared_error']
#val_mse_hist = history.history['val_mean_squared_error']


#for round 2 model:
#model = load_model(filename_base + '_fold_' + str(i) + '_round2.h5')
model = load_model(filename_base + '_fold_' + str(i) + '.h5')  #, custom_objects={'AdamW':adamw})

preds_val_initial = model.predict(X_val_list_scaled, verbose=1, batch_size=256)
preds_val_initial = preds_val_initial.reshape(preds_val_initial.shape[0])
    
    #****
#y_val_list_scaled = y_val_list[0]
#preds_val = preds_val_initial

y_val_list_scaled = y_val_list[i] #np.divide((y_val_list_scaled), 4)
preds_val = np.divide((preds_val_initial), 4)
    
    #y_train_list_scaled = np.divide((y_train_list[0]-18),41) - 1
    
    #preds_val = np.multipy((preds_val_initial), 82) + 18
    #y_train_list_scaled = np.multipiy((y_train_list_scaled), 82) +18

spearman = stats.spearmanr(preds_val, y_val_list_scaled)
pearson = stats.pearsonr(preds_val, y_val_list_scaled)
    
mean_ae = np.mean(abs(preds_val - y_val_list_scaled))
median_ae = np.median(abs(preds_val - y_val_list_scaled))
    
print(pearson)
print(mean_ae)
print(median_ae)


##best_epochs.append(best_epoch)
#mses.append(mse_model)
##val_mses.append(val_mse_model)
#mse_hists.append(mse_hist)
#val_mse_hists.append(val_mse_hist)
#spearmans.append(spearman)
#pearsons.append(pearson)
#mean_aes.append(mean_ae)
#median_aes.append(median_ae)

In [ ]:

#y_val_list_scaled

In [ ]:
print('Best epoch: ', best_epochs)
print('MSE: ', mses)
print('Val MSE: ', val_mses)
print('Spearman: ', spearmans)
print('Pearson: ', pearsons)
print('Mean abs error: ', mean_aes)
print('Median abs error: ', median_aes)


#print('Mean MSE: ', np.mean(mses))
#print('Mean val_MSE: ', np.mean(val_mses))
#print('Mean Spearman r: ', [np.mean(i) for i in zip(*spearmans)])
#print('Mean Pearson r: ', [np.mean(i) for i in zip(*pearsons)])
#print('Mean of Mean abs error: ', np.mean(mean_aes))
#print('Mean of Median abs error: ', np.mean(median_aes))


mean_mse = [np.mean(i) for i in zip(*mse_hists)]
mean_val_mse = [np.mean(i) for i in zip(*val_mse_hists)]
plt.plot(mean_mse[3:])
plt.plot(mean_val_mse[3:])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

#for j in range(len(mse_hists)):
#    plt.plot(mse_hists[j])
#    plt.plot(val_mse_hists[j])
#    plt.show()

In [ ]:
plt.plot(mean_mse[50:1000])
plt.plot(mean_val_mse[50:1000])

In [ ]:
#for PR, max limit 420, for age 105

fig, ax = plt.subplots(figsize=(12,12)) 
plt.scatter(y_val_list_scaled, preds_val, alpha=0.3)
    
ax.set_title('Age Using MGH 12-Lead ECGs', fontsize=28)

ax.set(xlim = [0, 105], ylim = [0, 105])
plt.xticks(range(0, 105, 10), fontsize=20)
plt.yticks(range(0, 105, 10), fontsize=20)
plt.xlabel('Actual Age', fontsize=26)
plt.ylabel('Predicted Age', fontsize=26)

ax.plot([0, 105], [0,105], alpha=0.2)
plt.show()

In [ ]:
y_val_list_scaled = np.multiply((y_val_list[i]), 4)
model = load_model(filename_base + '_fold_' + str(i) + '.h5')

onecyc = OneCycle(lr_range=(0.0005, 0.005), momentum_range=(0.95,0.85))
    
earlystopper = EarlyStopping(monitor = 'val_mean_squared_error', mode='min', patience=num_patience, verbose=1)  #restore_best_weights not available
checkpointer = ModelCheckpoint(filename_base + '_fold_' + str(i) + '_round2.h5', monitor = 'val_mean_squared_error', mode='min', verbose=1, save_best_only = True)

history = model.fit(X_train_list_scaled, y_train_list_scaled, validation_data = (X_val_list_scaled, y_val_list_scaled), batch_size = num_batch_size, epochs=num_epochs, callbacks=[checkpointer, onecyc, earlystopper])   #[corr_check, onecyc, early_stopper]) #tensorboard_callback])
    